In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.sparse import csc_matrix
from scipy.sparse import csr_matrix

In [22]:
file  = np.load('user_movie_rating.npy')
user_id = file[:,0]
movie_id = file[:,1]
rating = file[:,2]

count_ones = np.sum(user_id == 1)
count_twos = np.sum(user_id == 2)
print(count_ones)

similarity_threshold = 0.5
print(len(movie_id))
print(movie_id[count_ones+count_twos-1:])
print(file)

626
65225506
[17764    30    84 ... 17622 17627 17764]
[[     1     30      3]
 [     1    157      3]
 [     1    173      4]
 ...
 [103703  17622      2]
 [103703  17627      4]
 [103703  17764      4]]


In [ ]:
#create a sparse matrix of the user-movie rating, where the movies that recieved a rating get a 1 and the ones that did not get a rating get a 0
#We assume from the assignment that we only care about movies that were rated but not the actual ratings. Thus giving a binary matrix

def create_matrix(file):
    user_id = file[:,0]
    movie_id = file[:,1]
    rating = file[:,2]
    data = np.ones(len(rating))
    user_movie_matrix = csc_matrix((data, (movie_id, user_id))) #heb movie en user omgedraaid, omdat we de users als colommen willen hebben
    return user_movie_matrix

In [ ]:
user_movie_matrix = create_matrix(file)
#delete the first row and column since they are empty
user_movie_matrix = user_movie_matrix[1:,1:]
# print(user_movie_matrix)
# print(user_movie_matrix.shape)

print(user_movie_matrix.toarray()[27:31,0:3]) #to see the axes of the matrix

[[0. 1. 0.]
 [0. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]


Note: Ik heb nu de binary matrix getransposed om de de users als kolommen en movies als rijen willen.

In [ ]:
#make a minhash function for the csc matrix, I realise now that a csr matrix might have been more efficient
def minhash(data, num_permutations):
    if not isinstance(data, csc_matrix):
        raise ValueError("Input data must be a scipy.sparse CSC matrix.")
    
    num_movies = data.shape[1]  # Number of columns (features)
    num_users = data.shape[0]   # Number of rows (samples)
    
    # Create random permutations
    permutations = [np.random.permutation(num_movies) for _ in range(num_permutations)]
    perm_signature = np.full((num_users, num_permutations), np.inf)
    
    # Iterate over columns (movies/features) of the CSC matrix
    for movie_index in range(num_movies):
        # Get all non-zero row indices for the current column
        row_indices = data.indices[data.indptr[movie_index]:data.indptr[movie_index + 1]]
        
        for k in range(num_permutations):
            # Compute the hash value for the current column (movie_index)
            hash_value = permutations[k][movie_index]
            
            # Update the MinHash signature for the rows with non-zero values in this column
            for row_index in row_indices:
                perm_signature[row_index, k] = min(perm_signature[row_index, k], hash_value)

    return perm_signature#.transpose()



In [ ]:
testhash = minhash(user_movie_matrix, 100)

# #save the minhash signature to a file
np.save('minhash_transposed.npy', testhash)
testhash = np.load('minhash_transposed.npy')
print(testhash)
print(testhash.shape)


KeyboardInterrupt: 

In [38]:
print(testhash)
print(testhash.shape)

[[ 54.  40.  40. ...  40.  16.  40.]
 [ 32.   9.   9. ...  23.  13.  32.]
 [ 30.  92.  11. ... 514.  92. 200.]
 ...
 [  4.   3.   3. ...   3.   3.   3.]
 [  7.   7.  10. ... 139.  31. 227.]
 [  6.   6.  27. ...  48.  71.  62.]]
(100, 103703)


In [ ]:
#print the first column from the user_movie_matrix, so for the first person
print(user_movie_matrix[0,:])
print(testhash[:,0])

[ 54.  32.  30.  44.  31.  33.  16.   4. 106.   1.   1.   6.  65.  19.
  27.  42.  93.  12.   9.   5.  23.   3.   8.  10.   3.  14.   6.  23.
  54.  10.   3.  98.   9.  78.  10.   5.  17.  34.  11.  16.   2.  17.
  12.  17.   5.  23.  99.  13.  10.   6.   6.  19.   7.  28.  20.  28.
  40.   5.  14.  15.   3.  12.   2.  32.   1.  10.  41.  82.  96.  25.
  16.  62.  30.  15.  21.  10.  10.  46.  83.  12.  20.  75.   1.  13.
  20.  18.  66.  12.  10.  21.  16.  23.   8.  64.  42.  23.  89.   4.
   7.   6.]


In [ ]:
def jaccard_similarity(signature1, signature2):
    return np.sum(signature1 == signature2) / np.sum(signature1 != np.inf)

def lsh(signatures, num_hashes, similarity_threshold):
    num_bands = 50
    band_size = num_hashes // num_bands

    buckets = {}
    for user_id, signature in enumerate(signatures):
        for band in range(num_bands):
            # Group the portion of the signature belonging to this band
            band_hash = tuple(signature[band * band_size:(band + 1) * band_size])
            if band_hash not in buckets:
                buckets[band_hash] = []
            buckets[band_hash].append(user_id)

    candidate_pairs = set()
    for bucket_users in buckets.values():
        for i in range(len(bucket_users)):
            for j in range(i + 1, len(bucket_users)):
                candidate_pairs.add((bucket_users[i], bucket_users[j]))
    
    similar_users = []
    for user1, user2 in candidate_pairs:
        sim = jaccard_similarity(signatures[user1], signatures[user2])
        if sim >= similarity_threshold:
            similar_users.append((user1, user2))

    return similar_users

In [ ]:
similar_users = lsh(testhash, 100, 0.5)
print(similar_users)

#todo check what exactly the numbers are, are they the user id's or movie id's or what are they and why are they the same number.
# It could have something to do with the fact that when I ran this code I had not yet transposed the minhash signature,
# so the users and movies were switched.

[(25, 25), (59, 59), (0, 0), (92, 92), (33, 33), (66, 66), (7, 7), (99, 99), (40, 40), (73, 73), (14, 14), (47, 47), (80, 80), (21, 21), (54, 54), (13, 13), (87, 87), (46, 46), (28, 28), (61, 61), (20, 20), (2, 2), (94, 94), (35, 35), (68, 68), (9, 9), (42, 42), (75, 75), (34, 34), (16, 16), (67, 67), (49, 49), (8, 8), (82, 82), (41, 41), (23, 23), (74, 74), (56, 56), (15, 15), (89, 89), (48, 48), (30, 30), (81, 81), (22, 22), (55, 55), (63, 63), (88, 88), (70, 70), (29, 29), (62, 62), (44, 44), (3, 3), (95, 95), (36, 36), (96, 96), (18, 18), (69, 69), (10, 10), (43, 43), (76, 76), (77, 77), (51, 51), (17, 17), (50, 50), (83, 83), (24, 24), (84, 84), (57, 57), (58, 58), (90, 90), (31, 31), (91, 91), (32, 32), (64, 64), (5, 5), (65, 65), (97, 97), (6, 6), (98, 98), (39, 39), (72, 72), (4, 4), (38, 38), (11, 11), (71, 71), (12, 12), (45, 45), (78, 78), (19, 19), (79, 79), (52, 52), (85, 85), (53, 53), (26, 26), (86, 86), (27, 27), (60, 60), (1, 1), (93, 93)]


In [49]:
print(len(similar_users))

99
